In [1]:
# Importações necessárias
import asyncio
from telethon.sync import TelegramClient # Embora o nome seja 'sync', ele opera no loop asyncio
from telethon.errors.rpcerrorlist import SessionPasswordNeededError, UserDeactivatedBanError, AuthKeyUnregisteredError
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerChannel
from datetime import datetime, timedelta
import os, json

# Para permitir que o loop de eventos do asyncio funcione dentro do Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

# --- Configurações Essenciais ---
API_ID = 6942037  # Coloque seu API ID aqui (número inteiro)
API_HASH = 'c773a928384aa55437efad4823467f32'  # Coloque seu API Hash aqui (string)

# Nome da sua sessão existente (o nome do arquivo .session sem a extensão)
# Ex: se seu arquivo é 'minha_conta.session', coloque 'minha_conta'
SESSION_NAME = '+5571987946284' # Ou o nome que você usou antes

# Identificador do canal PÚBLICO para teste (ex: '@TelegramBR')
CHANNEL_IDENTIFIER = '@activtradespt'

# Seu número de telefone (importante para a primeira autenticação ou se a sessão expirar)
PHONE_NUMBER = '+5571987946284' # Ex: +55719XXXXXXXX
# --- Fim das Configurações ---

print("Configurações carregadas e nest_asyncio aplicado.")

Configurações carregadas e nest_asyncio aplicado.


In [2]:
async def get_all_yesterday_messages():
    yesterday_obj = datetime.now() - timedelta(days=1)
    yesterday_date_val = yesterday_obj.date()
    texts_for_yesterday = []

    async with TelegramClient(SESSION_NAME, API_ID, API_HASH) as client:
        channel_entity = await client.get_entity(CHANNEL_IDENTIFIER)
        
        # client.iter_messages iterates from newest to oldest by default.
        # We'll iterate and collect messages from yesterday, stopping when we pass into the day before.
        # limit=None means iterate as many messages as needed.
        async for msg in client.iter_messages(channel_entity, limit=None):
            # Ensure msg.date is timezone-aware or convert to naive if comparing with naive yesterday_date_val
            # Telethon messages usually have timezone-aware UTC datetimes.
            current_msg_date = msg.date.date()

            if current_msg_date == yesterday_date_val:
                if msg.text: # Using msg.text for cleaner text, msg.message is also an option
                    texts_for_yesterday.append(msg.text)
            elif current_msg_date < yesterday_date_val:
                # We've iterated past yesterday's messages (into older days)
                break 
                
    return texts_for_yesterday

# To use this function:
texts = await get_all_yesterday_messages()
print(json.dumps(texts, ensure_ascii=False))


["Clique aqui e não perca: https://www.youtube.com/live/DgzB0PdDyLk?si=hfRTSmmrRZ1Y-DmA", "As Maiores - S&P500/Nasdaq", "As Maiores - Ibovespa", "Snapshot de Fechamento", "Leia a análise completa: https://br.tradingview.com/chart/GOLD/I4fVv2eL/", "**🔴 Em 15 Minutos\n\nAcompanhe Ao Vivo o Fechamento de Mercado com Valber Bastos da Macro Trader (17:30)**\n\nhttps://youtube.com/live/VsoNKc4n1O4?feature=share", "**Os índices de ações americanas seguem ampliando alta na sessão e renovam máxima do dia**\n  \n  __Via Terminal Macro Trader / ActivTrades\n\n...\nDisclaimer ActivTrades Brasil: Os derivativos financeiros são instrumentos complexos e apresentam um alto risco de perder dinheiro rapidamente devido à alavancagem. 84% das contas de investidores de retalhos perdem dinheiro quando negociam derivativos__", "**FX no Radar: Emergente**\n    \n**Mesmo com o dólar forte, o yuan offshore (CNH) mostrou resistência após dados da China mostrarem inflação alinhada às expectativas**\n  \n▶️ O índi

In [3]:
texts[2]

'As Maiores - Ibovespa'

### Também criar cache para evitar carregar o Telegram

## Fazer limpeza e sumarizar o texto


In [5]:
# summarize.py

import sys
import json
import re # limpeza do texto
from langchain.docstore.document import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv
from pathlib import Path
import os

# Sempre procura o .env na raiz
load_dotenv(".env")

API_ID = os.getenv("TELEGRAM_API_ID")
API_HASH = os.getenv("TELEGRAM_API_HASH")
CHANNEL_IDENTIFIER = os.getenv("TELEGRAM_CHANNEL")
SESSION_NAME = os.getenv("TELEGRAM_SESSION_NAME")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# 🧠 Inicializa o LLM da Google com LangChain
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

In [26]:
# 🛠️ Usa a chain de resumo da LangChain
summary_prompt = PromptTemplate.from_template("""
Você é um analista financeiro e deve resumir as principais notícias do dia anterior sobre mercados, política e empresas.
Organize em tópicos, destaque os pontos mais relevantes e elimine repetições e termos promocionais.

Texto:
{input}

Resumo claro, direto e com tópicos:
""")

summarize_chain = summary_prompt | llm


def clean_text(text):
    # Remove disclaimers repetitivos
    text = re.sub(r'Disclaimer ActivTrades.*?derivativos__', '', text, flags=re.DOTALL)
    # Remove links
    text = re.sub(r'https?://\S+', '', text)
    # Remove espaçamentos excessivos
    text = re.sub(r'\n{2,}', '\n', text)
    return text.strip()

def summarize(texts):
    # 🧾 Cria o documento para a chain
    content = "\n\n".join([clean_text(t) for t in texts])
    # documents = [Document(page_content=content)]

    # 📄 Executa o resumo
    result = summarize_chain.invoke({"input": content})
    if hasattr(result, "content"):
        summary = result.content
    else:
        summary = result
    return summary


In [27]:

result = summarize(texts)
print(result)


## Resumo do Dia Anterior – Mercados, Política e Empresas

**Mercados Globais:**

*   **EUA-China:** Acordo para suspender temporariamente tarifas bilaterais por 90 dias impulsionou mercados. EUA reduzirão tarifas sobre produtos chineses de 145% para 30%, China de 125% para 10%. Setores de aço, alumínio, farma e fentanil não foram incluídos no acordo.
*   **Ações:** Índices de ações globais em alta, com destaque para as Big Techs americanas na abertura (Amazon liderando). Ibovespa futuro abriu com gap de alta, mas perdeu força ao longo do dia.
*   **Câmbio:** Dólar em alta frente aos principais pares e misto contra emergentes.
*   **Petróleo:** Preços em alta com otimismo sobre acordo EUA-China.
*   **Juros:** Expectativas para cortes do Fed reajustadas.
*   **Metais:** Ouro em queda, influenciado pela diminuição da aversão ao risco.
*   **Boletim Focus:** Projeções para a inflação de 2025 recuando pela quarta semana consecutiva.

**Política:**

*   **EUA-China:** Expectativa de novas 

In [28]:
type(result)

str

In [11]:
def format_markdown(summary):
    return "\n".join([
        f"### 📌 {line.strip()}" if line.startswith("*") or line.startswith("▶️") else line
        for line in summary.split("\n") if line.strip()
    ])

In [29]:
print(format_markdown(result))

## Resumo do Dia Anterior – Mercados, Política e Empresas
### 📌 **Mercados Globais:**
### 📌 *   **EUA-China:** Acordo para suspender temporariamente tarifas bilaterais por 90 dias impulsionou mercados. EUA reduzirão tarifas sobre produtos chineses de 145% para 30%, China de 125% para 10%. Setores de aço, alumínio, farma e fentanil não foram incluídos no acordo.
### 📌 *   **Ações:** Índices de ações globais em alta, com destaque para as Big Techs americanas na abertura (Amazon liderando). Ibovespa futuro abriu com gap de alta, mas perdeu força ao longo do dia.
### 📌 *   **Câmbio:** Dólar em alta frente aos principais pares e misto contra emergentes.
### 📌 *   **Petróleo:** Preços em alta com otimismo sobre acordo EUA-China.
### 📌 *   **Juros:** Expectativas para cortes do Fed reajustadas.
### 📌 *   **Metais:** Ouro em queda, influenciado pela diminuição da aversão ao risco.
### 📌 *   **Boletim Focus:** Projeções para a inflação de 2025 recuando pela quarta semana consecutiva.
### 📌 **Po

## Testar o acesso a api


In [17]:
import requests

In [24]:
API_URL = "http://localhost:8080/resumo"  # ou a URL do seu deploy Railway

def format_markdown(summary):
    return "\n".join([
        f"### 📌 {line.strip()}" if line.startswith("*") or line.startswith("▶️") else line
        for line in summary.split("\n") if line.strip()
    ])

response = requests.get(API_URL, timeout=30)
data = response.json()
resumo = data.get("resumo", "")

    

In [25]:
data.get("resumo").

'content=\'Aqui está um resumo das principais notícias do dia anterior, organizadas por tópico:\\n\\n**Estados Unidos e China:**\\n\\n*   **Redução de Tarifas:** EUA e China concordaram em reduzir tarifas por 90 dias. As tarifas dos EUA sobre produtos chineses caem de 145% para 30%, enquanto as tarifas chinesas sobre produtos americanos diminuem de 125% para 10%.\\n*   **Fentanil:** As tarifas dos EUA sobre o fentanil permanecem em vigor, separadas do acordo geral de redução de tarifas.\\n*   **Próximos Passos:** EUA e China planejam se reunir novamente em breve para discutir acordos mais amplos e barreiras não tarifárias.\\n*   **Impacto:** O alívio nas tensões comerciais deve impulsionar o comércio, mas analistas alertam sobre incertezas persistentes.\\n\\n**Política dos EUA:**\\n\\n*   **Preços de Medicamentos:** Trump prometeu reduzir os preços dos medicamentos em até 59% e assinou uma ordem executiva com esse objetivo. Ações de empresas farmacêuticas caíram. A Casa Branca mira med

In [19]:
format_markdown(resumo)

'content=\'## Resumo do Dia Anterior: Mercados, Política e Empresas\\n\\n**Destaque Principal: Acordo EUA-China**\\n\\n*   **Redução de Tarifas:** EUA e China concordaram em reduzir tarifas por 90 dias. EUA cortam de 145% para 30% sobre produtos chineses, enquanto a China reduz de 125% para 10% sobre produtos americanos (exceto tarifas relacionadas ao fentanil).\\n*   **Impacto no Mercado:** Bolsas globais reagiram positivamente, com alta nos índices de ações e queda no preço do ouro. Dólar se fortaleceu.\\n*   **Perspectivas:** Expectativa de aumento nos fluxos comerciais bilaterais, mas incertezas sobre a sustentabilidade do acordo a longo prazo. Próximas reuniões entre as partes devem buscar um acordo mais abrangente.\\n*   **Analistas:** Alívio nas tensões comerciais, mas ainda há incertezas para empresas e famílias.\\n\\n**Política e Geopolítica**\\n\\n*   **EUA x Irã:** Novas sanções dos EUA contra indivíduos e entidades iranianas ligadas à pesquisa de defesa.\\n*   **Ucrânia x R

✅ Próximos Passos
	•	Adicionar cache para evitar chamar a API desnecessariamente
	•	Exibir também as mensagens originais (com toggle)
	•	Permitir escolha do canal (multi-canal)
	•	Autenticar com senha simples (modo privado)

# Código com mais detalhes

In [13]:
async def get_yesterday_messages():
    yesterday = datetime.now() - timedelta(days=1)
    async with TelegramClient(SESSION_NAME, API_ID, API_HASH) as client:
        channel_entity = await client.get_entity(CHANNEL_IDENTIFIER)
        # messages = client.iter_messages(channel, offset_date=datetime.now(), reverse=True)
        history = await client(GetHistoryRequest(
            peer=channel_entity,
            offset_id=0,
            offset_date=datetime.now(),
            #reverse = True,
            add_offset=0,
            limit=0, # Número de mensagens a serem recuperadas
            max_id=0,
            min_id=0,
            hash=0
        ))
        
        texts = [msg.message for msg in history.messages if msg.date.date() == yesterday.date() and msg.message]
        return texts

texts = await get_yesterday_messages()
print(json.dumps(texts, ensure_ascii=False))

[]


/var/folders/68/xhdx3jc908q7wgf4wy53rvnw0000gn/T/ipykernel_59475/1000501570.py:21: RuntimeWarning: coroutine 'get_yesterday_messages' was never awaited
  texts = await get_yesterday_messages()


In [17]:
# Cria uma instância do cliente.
# O arquivo SESSION_NAME.session será procurado/criado no mesmo diretório do notebook,
# ou no caminho especificado em SESSION_NAME.
client = TelegramClient(SESSION_NAME, API_ID, API_HASH)

print(f"Cliente Telegram inicializado com o nome de sessão: '{SESSION_NAME}'")

Cliente Telegram inicializado com o nome de sessão: '+5571987946284'


In [18]:
try:
    if not client.is_connected():
        print("Tentando conectar ao Telegram...")
        await client.connect()
    else:
        print("Cliente já está conectado.")

    if not await client.is_user_authorized():
        print(f"Autenticação necessária para o número: {PHONE_NUMBER}")
        await client.send_code_request(PHONE_NUMBER)
        try:
            code = input('Digite o código de login recebido: ')
            await client.sign_in(PHONE_NUMBER, code)
        except SessionPasswordNeededError:
            password = input('Sua senha da autenticação de dois fatores (nuvem) é necessária: ')
            await client.sign_in(password=password)
        except Exception as e:
            print(f"Erro durante o sign_in: {e}")
            # Você pode querer parar aqui ou tentar desconectar
            raise # Re-levanta a exceção para parar a execução da célula se o login falhar
    
    print("Conectado e autorizado com sucesso!")

except AuthKeyUnregisteredError:
    print("ERRO CRÍTICO: A chave de autorização não está registrada (sessão inválida).")
    print(f"Por favor, delete o arquivo '{SESSION_NAME}.session' e rode esta célula novamente para criar uma nova sessão.")
except Exception as e:
    print(f"Ocorreu um erro ao conectar/autenticar: {e}")
    import traceback
    traceback.print_exc()

Tentando conectar ao Telegram...
Conectado e autorizado com sucesso!


Server closed the connection: [Errno 60] Operation timed out
Graceful disconnection timed out, forcibly ignoring cleanup
Attempt 1 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 2 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 3 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 4 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 5 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 6 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 1 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 2 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 3 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 4 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 5 at connecting failed: OSError: [Errno 51] Network is unreachable
Attempt 6 at connecting failed: OSError: [Errno 51] Ne

In [19]:

async def main():
    # Conectando ao Telegram
    await client.connect()

    # Autenticando, se necessário (código será enviado para o seu Telegram na primeira vez)
    if not await client.is_user_authorized():
        await client.send_code_request(phone_number)
        await client.sign_in(phone_number, input('Digite o código recebido: '))

    try:
        # Obtendo a entidade do canal
        channel_entity = await client.get_entity(channel_identifier)

        # Exemplo 1: Obter informações completas do canal
        full_channel_info = await client(GetFullChannelRequest(channel=channel_entity))
        print(f"Nome do Canal: {full_channel_info.chats[0].title}")
        print(f"Sobre: {full_channel_info.full_chat.about}")
        print(f"Participantes: {full_channel_info.full_chat.participants_count}")
        print("---")

        # Exemplo 2: Obter as últimas mensagens do canal
        print("Últimas mensagens:")
        # messages = await client.get_messages(channel_entity, limit=5) # Forma mais simples
        history = await client(GetHistoryRequest(
            peer=channel_entity,
            offset_id=0,
            offset_date=None,
            add_offset=0,
            limit=5, # Número de mensagens a serem recuperadas
            max_id=0,
            min_id=0,
            hash=0
        ))

        for message in history.messages:
            print(f"ID: {message.id} | Data: {message.date} | Remetente ID: {message.sender_id}")
            if message.message: # Verifica se há texto na mensagem
                print(f"  Texto: {message.message[:50]}...") # Mostra os primeiros 50 caracteres
            if message.media:
                print(f"  Mídia: {type(message.media).__name__}")
            print("---")

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
    finally:
        # Desconectando o cliente
        await client.disconnect()

In [13]:
channel_entity = None # Definir fora do try para que possa ser usado depois, se bem-sucedido
if client.is_connected() and await client.is_user_authorized():
    print(f"\nTentando acessar o canal: {CHANNEL_IDENTIFIER}")
    try:
        channel_entity = await client.get_entity(CHANNEL_IDENTIFIER)
        print(f"Canal encontrado: {getattr(channel_entity, 'title', 'Título não disponível')}")
        print(f"ID do Canal: {channel_entity.id}")
        print(f"Tipo: {type(channel_entity).__name__}")
    except ValueError:
        print(f"Erro: Canal '{CHANNEL_IDENTIFIER}' não encontrado ou identificador inválido. Verifique o nome/ID.")
    except UserDeactivatedBanError:
        print("Erro: A conta associada a esta sessão foi banida ou desativada.")
    except Exception as e:
        print(f"Erro ao tentar obter a entidade do canal: {e}")
else:
    print("Cliente não está conectado ou autorizado. Execute a célula de conexão/autenticação primeiro.")


Tentando acessar o canal: @activtradespt
Canal encontrado: ActivTrades Português
ID do Canal: 1257215455
Tipo: Channel


In [20]:
messages = [] # Definir fora do try para que possa ser usado depois
if channel_entity: # Verifica se channel_entity foi obtido com sucesso na célula anterior
    print(f"\nBuscando as últimas 5 mensagens do canal '{getattr(channel_entity, 'title', CHANNEL_IDENTIFIER)}'...")
    try:
        history = await client(GetHistoryRequest(
            peer=channel_entity,
            offset_id=0,
            offset_date=None,
            add_offset=0,
            limit=5, # Busque mais se precisar
            max_id=0,
            min_id=0,
            hash=0
        ))

        if not history.messages:
            print("Nenhuma mensagem encontrada no canal ou não foi possível acessá-las.")
        else:
            messages = history.messages # Armazena as mensagens para a próxima célula
            print(f"Encontradas {len(messages)} mensagens.")
            
    except Exception as e:
        print(f"Erro ao buscar mensagens: {e}")
        import traceback
        traceback.print_exc()
else:
    print("Entidade do canal não encontrada. Execute a célula anterior para obter as informações do canal.")


Buscando as últimas 5 mensagens do canal 'ActivTrades Português'...
Encontradas 5 mensagens.


In [26]:
messages[0].get_entities_text()

[(<telethon.tl.types.MessageEntityBold at 0x10b1f85a0>,
  'Tensão no Oriente Médio no Radar'),
 (<telethon.tl.types.MessageEntityBold at 0x10b1f86b0>,
  'Tensão entre Índia e Paquistão adia acordo comercial com a Índia, segundo executivos ligados à Casa Branca; Reino Unido assume prioridade'),
 (<telethon.tl.types.MessageEntityItalic at 0x10b1fcb00>,
  'Via Terminal Macro Trader / ActivTrades\n\n...\nDisclaimer ActivTrades Brasil: Os derivativos financeiros são instrumentos complexos e apresentam um alto risco de perder dinheiro rapidamente devido à alavancagem. 83% das contas de investidores de retalhos perdem dinheiro quando negociam derivativos')]

In [27]:
messages[0].message

'Tensão no Oriente Médio no Radar\n    \nTensão entre Índia e Paquistão adia acordo comercial com a Índia, segundo executivos ligados à Casa Branca; Reino Unido assume prioridade\n  \n  Via Terminal Macro Trader / ActivTrades\n\n...\nDisclaimer ActivTrades Brasil: Os derivativos financeiros são instrumentos complexos e apresentam um alto risco de perder dinheiro rapidamente devido à alavancagem. 83% das contas de investidores de retalhos perdem dinheiro quando negociam derivativos'

In [29]:
# Célula 6: Exibir o Conteúdo Detalhado das Mensagens (Modificada)

if messages: # Verifica se a lista de mensagens não está vazia
    print(f"\n--- Exibindo Conteúdo Detalhado de {len(messages)} Mensagens ---")
    for i, message in enumerate(messages):
        print(f"\n===== MENSAGEM {i+1} =====")
        print(f"ID da Mensagem: {message.id}")
        print(f"Data: {message.date.strftime('%Y-%m-%d %H:%M:%S %Z')}") # %Z para fuso horário, se disponível
        
        # Informações do Remetente
        sender_info = f"ID do Remetente: {message.sender_id}"
        if message.sender:
            if hasattr(message.sender, 'username') and message.sender.username:
                sender_info += f", Username: @{message.sender.username}"
            elif hasattr(message.sender, 'title'): # Para canais
                sender_info += f", Título do Canal: {message.sender.title}"
            elif hasattr(message.sender, 'first_name'): # Para usuários
                name_parts = [message.sender.first_name]
                if hasattr(message.sender, 'last_name') and message.sender.last_name:
                    name_parts.append(message.sender.last_name)
                sender_info += f", Nome: {' '.join(name_parts)}"
        print(sender_info)

        # Texto da Mensagem
        if message.message: # message.text é o texto simples, message.message pode incluir formatação
            print(f"Texto da Mensagem:\n---\n{message.message}\n---")
        else:
            print("Texto da Mensagem: (Nenhum texto)")

        # Mídia
        if message.media:
            media_type = type(message.media).__name__
            print(f"Mídia: Sim (Tipo: {media_type})")
            
            if media_type == 'MessageMediaPhoto':
                # Para fotos, você pode tentar acessar message.photo.id, message.photo.sizes, etc.
                print(f"  Detalhes da Foto: ID={message.photo.id}, Data={message.photo.date.strftime('%Y-%m-%d')}")
            elif media_type == 'MessageMediaDocument':
                doc_attrs = []
                if message.document.attributes:
                    for attr in message.document.attributes:
                        if type(attr).__name__ == 'DocumentAttributeFilename':
                            doc_attrs.append(f"Nome do Arquivo: {attr.file_name}")
                        if type(attr).__name__ == 'DocumentAttributeAudio':
                            doc_attrs.append(f"Duração: {attr.duration}s")
                            if attr.title: doc_attrs.append(f"Título: {attr.title}")
                            if attr.performer: doc_attrs.append(f"Artista: {attr.performer}")
                print(f"  Documento: MIME Type='{message.document.mime_type}', Tamanho={message.document.size} bytes")
                if doc_attrs: print(f"    Atributos: {'; '.join(doc_attrs)}")
            elif media_type == 'MessageMediaWebPage':
                if hasattr(message.media.webpage, 'url'):
                    print(f"  Preview da Webpage: URL='{message.media.webpage.url}'")
                if hasattr(message.media.webpage, 'site_name'):
                    print(f"    Site: {message.media.webpage.site_name}")
                if hasattr(message.media.webpage, 'title'):
                     print(f"    Título: {message.media.webpage.title}")
            # Adicione mais `elif` para outros tipos de mídia como MessageMediaPoll, MessageMediaGeo, etc.
        else:
            print("Mídia: Não")

        # Mensagem de Resposta
        if message.reply_to_msg_id:
            print(f"Em Resposta a: ID da Mensagem {message.reply_to_msg_id}")
            # Para buscar a mensagem original da resposta (requer outra chamada à API):
            replied_to_message = await client.get_messages(channel_entity, ids=message.reply_to_msg_id)
            if replied_to_message:
                print(f"  Texto da mensagem respondida: {replied_to_message.text[:50]}...")

        # Mensagem Encaminhada
        if message.fwd_from:
            fwd_info = "Mensagem Encaminhada de: "
            if message.fwd_from.from_id:
                # Tenta obter a entidade original do encaminhamento
                try:
                    fwd_entity_peer = message.fwd_from.from_id
                    # Note: message.fwd_from.from_id é um Peer (PeerUser, PeerChat, PeerChannel)
                    # Para obter o nome/username, você precisaria de client.get_entity(fwd_entity_peer)
                    # o que é uma chamada de API adicional. Por simplicidade, mostraremos o ID.
                    fwd_info += f"ID Peer: {fwd_entity_peer}"
                except Exception as e:
                    fwd_info += "(Não foi possível obter detalhes do remetente original)"

            elif message.fwd_from.from_name: # Para usuários que ocultaram suas contas ao encaminhar
                 fwd_info += f"Nome: {message.fwd_from.from_name}"
            if message.fwd_from.date:
                fwd_info += f" (Data Original: {message.fwd_from.date.strftime('%Y-%m-%d %H:%M:%S')})"
            print(fwd_info)
            if message.fwd_from.saved_from_peer: # Se foi encaminhada de 'Mensagens Salvas' ou outro chat
                print(f"  Encaminhada via Peer ID: {message.fwd_from.saved_from_peer}, Msg ID: {message.fwd_from.saved_from_msg_id}")


        # Visualizações
        if message.views is not None: # Checa se não é None, pois pode não estar presente
            print(f"Visualizações: {message.views}")

        # Reações
        if message.reactions:
            reaction_summary = []
            for reaction_count in message.reactions.results:
                # reaction_count.reaction é um objeto ReactionEmoji ou ReactionCustomEmoji
                # Vamos tentar pegar o emoticon se for um emoji padrão
                emoticon = ""
                if hasattr(reaction_count.reaction, 'emoticon'):
                    emoticon = reaction_count.reaction.emoticon
                elif hasattr(reaction_count.reaction, 'document_id'): # Para reações customizadas
                    emoticon = f"(Custom ID: {reaction_count.reaction.document_id})"
                
                reaction_summary.append(f"{emoticon}: {reaction_count.count}")
            if reaction_summary:
                print(f"Reações: {', '.join(reaction_summary)}")
        
        # Entidades (formatação, links, menções)
        if message.entities:
            entity_types = [type(entity).__name__ for entity in message.entities]
            print(f"Entidades de Formatação: Sim ({', '.join(list(set(entity_types)))})")
            # Para ver detalhes de cada entidade:
            # for entity in message.entities:
            #     print(f"  - {type(entity).__name__}: Offset={entity.offset}, Length={entity.length}")
            #     if hasattr(entity, 'url'): print(f"    URL: {entity.url}")
            #     if hasattr(entity, 'user_id'): print(f"    User ID: {entity.user_id}")


        # ID de Grupo (para álbuns de mídia)
        if message.grouped_id:
            print(f"ID do Grupo (Álbum): {message.grouped_id}")

        # Para explorar TODOS os campos da mensagem, descomente a linha abaixo:
        # print("\n--- Detalhes Completos (message.to_dict()) ---")
        # print(message.to_dict())
        # print("--- --- ---")

    print("\n===== FIM DA LISTA DE MENSAGENS =====")
else:
    print("Nenhuma mensagem para exibir. Execute a célula de busca de mensagens primeiro.")


--- Exibindo Conteúdo Detalhado de 5 Mensagens ---

===== MENSAGEM 1 =====
ID da Mensagem: 110278
Data: 2025-05-08 13:25:11 UTC
ID do Remetente: -1001257215455
Texto da Mensagem:
---
Tensão no Oriente Médio no Radar
    
Tensão entre Índia e Paquistão adia acordo comercial com a Índia, segundo executivos ligados à Casa Branca; Reino Unido assume prioridade
  
  Via Terminal Macro Trader / ActivTrades

...
Disclaimer ActivTrades Brasil: Os derivativos financeiros são instrumentos complexos e apresentam um alto risco de perder dinheiro rapidamente devido à alavancagem. 83% das contas de investidores de retalhos perdem dinheiro quando negociam derivativos
---
Mídia: Não
Visualizações: 185
Entidades de Formatação: Sim (MessageEntityItalic, MessageEntityBold)

===== MENSAGEM 2 =====
ID da Mensagem: 110277
Data: 2025-05-08 13:24:27 UTC
ID do Remetente: -1001257215455
Texto da Mensagem:
---
Reino Unido x EUA no Radar
    
Primeiro-ministro do Reino Unido, Keir Starmer, participará de anúncio

In [16]:
if client.is_connected():
    print("\nDesconectando o cliente...")
    try:
        await client.disconnect()
        print("Cliente desconectado com sucesso.")
    except Exception as e:
        print(f"Erro ao desconectar: {e}")
else:
    print("Cliente já está desconectado ou não foi conectado.")


Desconectando o cliente...
Cliente desconectado com sucesso.
